In [2]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [290]:
gname = "../../graphs/se_u_3/"
wmax = 1000
iu = false
n = 1000;
p = 0.05;

In [291]:
function wgen()
    if iu
        return 1 / rand(1:wmax)
    else
        return rand(1:wmax)
    end
end

wgen (generic function with 1 method)

In [292]:
a = randGraph(n, p=p, weightGen = wgen);
ceil(Int64,nnz(a)/2)

24359

In [293]:
tree1 = akpw(a);
str1 = compStretches(tree1,a);
data1 = sum(str1),mean(str1)

(171437.72007898672,0.17143772007898672)

In [294]:
tree2 = randishPrim(a);
str2 = compStretches(tree2,a);
data2 = sum(str2),mean(str2)

(524901.6700086298,0.5249016700086298)

In [295]:
tree3 = randishKruskal(a);
str3 = compStretches(tree3,a);
data3 = sum(str3),mean(str3)

(1.163708510495567e6,1.163708510495567)

In [296]:
truex = randn(n); truex = truex - mean(truex);
b = lap(a) * truex; b = b - mean(b);

In [297]:
writeToFile(gname * "graph.mtx", a);

In [298]:
writeToFile(gname * "tree1.mtx", tree1);
writeToFile(gname * "tree2.mtx", tree2);
writeToFile(gname * "tree3.mtx", tree3);

In [299]:
writeToFile(gname * "x.vec", truex);
writeToFile(gname * "b.vec", b);

In [300]:
output = []
push!(output, "A random graph with $(n) vertices and $(ceil(Int64,nnz(a)/2)) edges.")
push!(output, "")
push!(output, "The graph is generated in the following way:")
push!(output, "    - for each vertex, select n / $(1/p) random neighbors")
push!(output, "    - get rid of duplicate edges")
if iu
    push!(output, "    - weights are reals sampled uniformly from 1/[1..$(wmax)]")
else
    push!(output, "    - weights are integers sampled uniformly from [1..$(wmax)]")
end
push!(output, "")
push!(output, "There are three trees in the folder.")
push!(output, "")
push!(output, "Tree1 - total stretch = $(data1[1]), ave stretch = $(data1[2])")
push!(output, "Tree2 - total stretch = $(data2[1]), ave stretch = $(data2[2])")
push!(output, "Tree3 - total stretch = $(data3[1]), ave stretch = $(data3[2])")

desc = open(gname * "description.txt", "w")
for i in 1:length(output)
    println(desc, output[i])
end
close(desc)